# Intro
Everybody is talking about the so called "flattening" of the COVID-19 curve, but can we have some sort of rough perspective of when this will happen? Or worse, if it is going to happen at all?

This is a short study I did based on the available COVID-19 numbers. By no means, do I intend this to be an accurate predictor of the current situation. In fact, I sincereley hope that I have made a mistake and the real numbers are way smaller than the ones projected here. 

The only reason I am sharing this notebook with the open-source community is for learning. I have learned a few bits while working on it, and I hope that others will find something useful in it too.

There are a few descriptions, and assumptions I need to document here, which I promise to do in the coming days. Please, if you find aggravating erros or unclarity, do not hesitate to point those out to me.

In [0]:
!pip install chart_studio

In [0]:
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import requests
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression

# Main Parameters

In [0]:
country = "Germany"
days_ahead = 21

# Data and Cleanup / Preparation

In [0]:
raw_data = requests.get("https://pomber.github.io/covid19/timeseries.json").json()

In [0]:
cases = [
    {"date": record["date"], "cases": record["confirmed"]}
    for record in raw_data[country]
]

df = pd.DataFrame(cases)
df["date"] = pd.to_datetime(df["date"])
df["acceleration"] = df["cases"] / df["cases"].shift(1)


df["acceleration"].replace([np.inf, np.nan], df["acceleration"].median(), inplace=True)
df.fillna(0, inplace=True)
df["acceleration"].replace(np.inf, df["cases"], inplace=True)

# Methodology

My approach uses a linear regression model known as [Moving Linear Regression](https://www.daytrading.com/moving-linear-regression). In a series of steps, MLR will predict the *acceleration* of the exponential curve of the total cases, adjusting itself one step at a time. The change of acceleration (the ratio between the number of cases in one day, to that of the day before), can be expressed with a linear function. This is because, the rate of change of an exponential function (the total number of cases) is its derivative, i.e. can be expressed with a linear function.

In [145]:
df

,date,cases,acceleration
0,2020-01-22,0,1.134700
1,2020-01-23,0,1.134700
2,2020-01-24,0,1.134700
3,2020-01-25,0,1.134700
4,2020-01-26,0,1.134700
...,...,...,...
64,2020-03-26,43938,1.177237
65,2020-03-27,50871,1.157791
66,2020-03-28,57695,1.134143
67,2020-03-29,62095,1.076263


In [0]:
def predict(df, n=30):
    pred = calculate_next(df["acceleration"], n)
    date = df.iloc[-1]["date"] + pd.DateOffset(1)

    pred_total_cases = (df.iloc[-1]["cases"] * pred).round()

    # print(pred_new_recoveries)

    return df.append(
        pd.DataFrame(
            [{"date": date, "cases": pred_total_cases, "acceleration": pred,}]
        ),
        ignore_index=True,
        sort=False,
    )


def calculate_next(data, n=30):
    r = LinearRegression()
    y = data.tail(n).values.reshape(-1, 1)
    r.fit(X=np.array(range(0, len(y))).reshape(-1, 1), y=y)

    return r.predict(np.array([[n + 1]]))[0][0]

In [0]:
for i in range(0, days_ahead):
  df = predict(df, n=21)

In [142]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["cases"].round(),
        mode="lines+markers",
        line_shape="spline",
        name="Actual # total cases",
    )
)
fig.add_trace(
    go.Scatter(
        x=df["date"].tail(days_ahead),
        y=df["cases"].round().tail(days_ahead),
        mode="lines+markers",
        line_shape="spline",
        name="Predicted # total cases",
    )
)

annotations = []
annotations.append(
    {
        "xref": "paper",
        "yref": "paper",
        "x": 0.0,
        "y": 1.05,
        "xanchor": "left",
        "yanchor": "bottom",
        "text": f"Actual vs Predicted Number of COVID-19 Total Cases in {country}",
        "font": {"family": "Arial", "size": 24, "color": "rgb(37,37,37)"},
        "showarrow": False,
    }
)
fig.update_layout(annotations=annotations)

fig.show()